In [1]:
import os
import openai
from dotenv import load_dotenv, find_dotenv
import requests
import json
from requests_html import HTMLSession
from bs4 import BeautifulSoup as bs

In [2]:
_ = load_dotenv(find_dotenv()) # read local env
# Set Openai API key
openai.api_key = os.environ['OPENAI_API_KEY']

In [3]:
def get_completion_from(messages, model='gpt-3.5-turbo', temperature=0, max_tokens=500):
    '''
    Openai chat completion request
    returns: string 
    '''
    response = openai.ChatCompletion.create(model=model, messages=messages, temperature=0, max_tokens=max_tokens)
    return response.choices[0].message['content']

In [4]:
def scrape_info_from(url: str) -> str:
    ''' 
    Scrape information from url provided
    returns: string
    '''
    s = HTMLSession().get(url)
    divp = s.html.find('h2,h3,h4,h5,h6,p,a,span')
    return '\n'.join([d.text for d in divp])

In [5]:
class MessageBuffer():
    
    def __init__(self, size=100):
        ''' 
        Message Buffer to limit the number of prompts saved in memory.
        Initializes size.
        Returns: MessageBuffer Object
        '''
        self.size=size
        self.messages= []
    
    def add(self, item):
        if len(self.messages)<self.size:
            self.messages.append(item)
        else:
            self.messages.pop(0)
            self.messages.append(item)
            
    def clean(self):
        self.messages=[]

In [8]:
class ChatBot(object):
    def __init__(self, name='ChipChat', model='gpt-3.5-turbo-16k', temperature=0, max_tokens=500, mode='commercial', buffer_size=10):
        self.name = name
        self.mode = mode
        self.model = model
        self.temperature = temperature
        self.max_tokens = max_tokens
        self.history = MessageBuffer(size=buffer_size)
        self.knowledge_base=[]
        self.system = 'You are a Business representative for industry 4.0 and you work for Critical Manufacturing, the best company ever! \
                        You only answer to questions that relate with the company business. \
                        If the question is not related with Critical manufacturing, semiconductor industry, electronics assembly \
                        industry or medical devices industry, just respond with this sentence between ###: \
                        ###Sorry, my knowledge is limited.### Your name is ChipChat. Your are funny and polite and very enthusiastic! \
                        Every time you are asked for future events, complement the answer by inviting them to join us!'
        self._get_links_list('https://www.criticalmanufacturing.com')
        
    
    def _get_links_list(self, url: str) -> list:
        self.knowledge_base = HTMLSession().get(url).html.links
        

    def get_completion_from(self, messages):
        '''
        Openai chat completion request
        returns: string 
        '''
        response = openai.ChatCompletion.create(model=self.model, messages=messages, temperature=self.temperature, max_tokens=self.max_tokens)
        return response.choices[0].message['content']
    
    
    def greetings(self):
        ''' 
        Say hello and introduces himself
        return: string 
        '''
        self.history.add({'role': 'system', 'content': self.system})
        self.history.add({'role':'user', 'content':'Present yourself'})
        
        # chat completion request
        response=get_completion_from(self.history.messages)
        # add response to messages history
        self.history.add({'role':'assistant', 'content': response})
        return response
    
    
    def extract_topic(self, prompt: str) -> (bool, str, str):
        system_message = {'role':'system', 'content':f' extract the 3 most common topics maximum in json format that are present in the user query, from the following:\
                                              weather, job openings, product functionalities, who we are, leadership, research, customers, services, news, events, gartner, industries, contact, goodbye. if no topic exists return as other \
                                              For instance, topics=[boda]'}
        messages = [system_message, {'role':'user', 'content':prompt}]
        response = get_completion_from(messages)
        return response
    
    
    def get_response_from(self, topic: str, question:str) -> str:
        messages = []
        if topic == 'news':
            links = [link for link in list(self.knowledge_base) if (topic in link) or ('press' in link)]
        elif topic == 'job-openings':
            links = ['https://careers.criticalmanufacturing.com/', 'https://careers.criticalmanufacturing.com/#jobs']
        else:
            links = [link for link in list(self.knowledge_base) if topic in link]
        context = '\n'.join([scrape_info_from(link) for link in links])
        if len(context)>60000:
            context = context[:60000]
        query = f""" Use the text below about Critical Manufacturing to answer the subsequent question. \ 
                    If the answer cannot be found, write "sorry friend, I don't know"

                    Text:
                    \"\"\"
                    {context}
                    \"\"\"

                    Question: {question}
                    """
        messages += self.history.messages
        messages.append({'role':'user', 'content':query})
        return self.get_completion_from(messages)
    
    
    def get_response(self, prompt: str):
        # add user prompt to chat history
        self.history.add({'role':'user', 'content': prompt})
        
        # Extract the most relevant topic 
        res = self.extract_topic(prompt)
        try:
            topic = json.loads(res)['topics'][0]
        except json.decoder.JSONDecodeError:
            topic = 'other'
        
        # find appropriate info for each topic
        match topic:
            case 'who we are':
                response = self.get_response_from(topic.replace(' ', '-'), prompt)
                self.history.add({'role':'assistant', 'content': response})
            case 'job openings':
                response = self.get_response_from(topic.replace(' ', '-'), prompt)
                self.history.add({'role':'assistant', 'content': response})
            case 'product functionalities':
                response = self.get_response_from('mes-for-industry-4-0', prompt)
                self.history.add({'role':'assistant', 'content': response})
            case 'leadership':
                response = self.get_response_from(topic, prompt)
                self.history.add({'role':'assistant', 'content': response})
            case 'research':
                response = self.get_response_from(topic, prompt)
                self.history.add({'role':'assistant', 'content': response})
            case 'customers':
                response = self.get_response_from(topic, prompt)
                self.history.add({'role':'assistant', 'content': response})
            case 'services':
                response = self.get_response_from(topic, prompt)
                self.history.add({'role':'assistant', 'content': response})
            case 'news':
                response = self.get_response_from(topic, prompt)
                self.history.add({'role':'assistant', 'content': response})
            case 'events':
                response = self.get_response_from(topic, prompt)
                self.history.add({'role':'assistant', 'content': response})
            case 'gartner':
                response = self.get_response_from(topic, prompt)
                self.history.add({'role':'assistant', 'content': response})
            case 'industries':
                response = self.get_response_from(topic, prompt)
                self.history.add({'role':'assistant', 'content': response})
            case 'contact':
                response = self.get_response_from(topic, prompt)
                self.history.add({'role':'assistant', 'content': response})
            case _:
                response = get_completion_from(self.history.messages)
                self.history.add({'role':'assistant', 'content': response})
        return response.replace('#', '').strip()
        
    def run():
        pass
        

#chip = ChatBot()
#print(chip.greetings())   

In [9]:
import panel as pn  # GUI
pn.extension()

started = False
panels = []

chipchat = ChatBot()

def collect_messages(_):
    global chipchat
    global started
    
    if started:
        prompt = user_input.value_input
        user_input.value = ''
        try:
            response = chipchat.get_response(prompt) 
        except openai.error.ServiceUnavailableError:
            response = 'Sorry, but it seems my knowledge is limited right now, try again later.'
        panels.append(
            pn.Row('User:', pn.pane.Markdown(prompt, width=600)))
        panels.append(
            pn.Row('Assistant:', pn.pane.Markdown(response, width=600, styles={'background-color': '#F6F6F6'})))
    else:
        started = True
        panels.append(pn.Row('Assistant:', pn.pane.Markdown(chipchat.greetings(), width=600, styles={'background-color': '#F6F6F6'})))
    
    return pn.Column(*panels)


user_input = pn.widgets.TextInput(value="", placeholder='Enter text here')
button_chat = pn.widgets.Button(name="Chat!")


conversation = pn.bind(collect_messages, button_chat)

dashboard = pn.Column(
    pn.Row(user_input, button_chat),
    pn.panel(conversation, loading_indicator=True, height=500),
)

dashboard

Column
    [0] Row
        [0] TextInput(placeholder='Enter text here')
        [1] Button(name='Chat!')
    [1] ParamFunction(function, _pane=Column, defer_load=False, height=500, loading_indicator=True)